# Library 설치

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import time, random
import pandas as pd
import json
import numpy as np
import re
from tqdm import tqdm
import math

## 옵션 설정

In [ ]:
# 소수점 아래 2자리까지 표시하도록 설정
pd.options.display.float_format = '{:.2f}'.format

# ... 드라이버 설정 부분 ...
chrome_options = Options()
# Chrome 주소창에 chrome://version을 입력하고 '프로필 경로'를 복사합니다. (.../User Data/Default 와 같은 경로)
# 프로필 경로에서 'Default'는 제외하고 'User Data'까지만 사용합니다.
# ❗(주의: 이 옵션을 사용하기 전에 모든 Chrome 창을 닫아야 합니다.)
user_data_path = "C:/Users/kbjoo/AppData/Local/Google/Chrome/User Data" 
chrome_options.add_argument(f"user-data-dir={user_data_path}")
# 실제 브라우저처럼 보이게 하는 User-Agent 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")

# 봇으로 감지될 가능성을 줄이는 추가 옵션들
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-gpu") # GPU 가속 비활성화 (일부 환경에서 안정성 향상)
chrome_options.add_argument("--no-sandbox") # 샌드박스 모드 비활성화 (리눅스 등 일부 환경에서 필요)
chrome_options.add_argument("--lang=ko_KR") # 브라우저 언어 설정
chrome_options.add_argument("window-size=1920x1080") # 실제 사용자와 유사한 창 크기 설정

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
except Exception as e:
    print(f"드라이버 초기화 중 오류 발생: {e}")
    exit()

In [ ]:
# --- 2. 수동 로그인 및 검색 페이지 이동 ---
# 먼저 네이버 메인 페이지로 이동하여 로그인
driver.get("https://www.naver.com")
time.sleep(random.uniform(2, 4))
# input("▶ 브라우저에서 네이버 로그인을 완료한 후, 터미널로 돌아와 Enter 키를 눌러주세요...")

# # 실제 검색을 원하는 페이지로 이동
# search_url = "https://search.shopping.naver.com/ns/search?query=탈모" # 예시 URL
# driver.get(search_url)
# print("검색 페이지로 이동했습니다. 스크롤을 시작합니다.")
# time.sleep(random.uniform(3, 5))

# 3. 검색창을 찾아 검색어 입력
search_keyword = "탈모"
search_box_before = driver.find_element(By.CSS_SELECTOR, 'input#input_text')
search_box_before.click()
time.sleep(random.uniform(1, 2))

search_box_before = driver.find_element(By.CSS_SELECTOR, 'input#input_text')
search_box_before.send_keys(search_keyword)
search_box_before.send_keys(Keys.ENTER)
time.sleep(random.uniform(3, 5))
print("검색 페이지로 이동했습니다. 스크롤을 시작합니다.")

검색 페이지로 이동했습니다. 스크롤을 시작합니다.


## 함수 세팅

In [ ]:
def human_like_scroll(driver, total_scroll_time=30):
    """
    조금 더 사람처럼 행동하는 스크롤 함수입니다.
    주로 아래로 스크롤하지만, 가끔씩 위로 스크롤하는 동작을 포함합니다.
    
    Args:
        driver: Selenium 웹 드라이버
        total_scroll_time (int): 이 시간(초) 동안 스크롤을 시도합니다.
    """
    start_time = time.time()
    
    while time.time() - start_time < total_scroll_time:
        # 10% 확률로 위로 스크롤
        if random.random() < 0.1:
            scroll_y = random.randint(-400, -150) # 위로 스크롤할 픽셀
            direction = "위로"
        # 90% 확률로 아래로 스크롤
        else:
            scroll_y = random.randint(500, 900) # 아래로 스크롤할 픽셀
            direction = "아래로"
            
        driver.execute_script(f"window.scrollBy(0, {scroll_y});")
        print(f"{direction} {abs(scroll_y)}픽셀 스크롤...")
        
        # 스크롤 후 랜덤한 시간 동안 대기
        time.sleep(random.uniform(1.5, 3.5))
        
        # 페이지 끝에 도달했는지 확인
        if driver.execute_script("return window.scrollY + window.innerHeight >= document.body.scrollHeight"):
            print("페이지 끝에 도달하여 스크롤을 종료합니다.")
            break
    else:
        print(f"설정된 시간({total_scroll_time}초)이 지나 스크롤을 종료합니다.")

In [ ]:
def smooth_scroll_by(driver, scroll_by_y, duration):
    """
    주어진 픽셀(scroll_by_y)만큼 주어진 시간(duration) 동안 부드럽게 스크롤합니다.
    """
    start_time = time.time()
    current_y = driver.execute_script("return window.scrollY")
    target_y = current_y + scroll_by_y

    while time.time() < start_time + duration:
        # 경과 시간을 기반으로 진행률 계산 (0.0 to 1.0)
        progress = (time.time() - start_time) / duration
        
        # 진행률이 1을 넘지 않도록 보정
        progress = min(progress, 1.0)

        # Ease-in-out 효과: 시작과 끝을 부드럽게 만듭니다.
        # sin 함수를 이용해 S자 곡선을 그려 가속/감속 효과를 냅니다.
        eased_progress = 0.5 * (1 - math.cos(progress * math.pi))
        
        # 현재 스크롤 위치 계산
        scroll_to = current_y + (scroll_by_y * eased_progress)
        driver.execute_script(f"window.scrollTo(0, {scroll_to})")
        
        # 브라우저가 렌더링할 시간을 주기 위한 짧은 대기
        time.sleep(0.01)

    # 오차 보정을 위해 스크롤이 끝나면 목표 지점으로 정확히 이동
    driver.execute_script(f"window.scrollTo(0, {target_y})")


In [ ]:
def advanced_smart_scroll(driver):
    """
    [최종 버전] 새로운 콘텐츠 로드를 보장하며, 사람처럼 부드러운 속도로 스크롤합니다.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤할 거리와 시간을 랜덤하게 설정
        scroll_distance = random.randint(800, 1200)
        scroll_duration = random.uniform(2.0, 3.5) # 2.0초 ~ 3.5초 사이

        print(f"{scroll_distance}px를 {scroll_duration:.1f}초 동안 부드럽게 스크롤합니다...")
        smooth_scroll_by(driver, scroll_distance, scroll_duration)
        
        # 새 콘텐츠가 로드될 시간을 충분히 줍니다.
        time.sleep(random.uniform(3.0, 5.0))

        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            print("페이지 높이가 더 이상 늘어나지 않아 스크롤을 종료합니다.")
            break
            
        last_height = new_height


# 상품 목록 수집

In [ ]:
# --- 3. 상품 목록 수집 ---

# 데이터를 담을 리스트
link_list, title_list, store_list, original_price_list, current_price_list, discount_rate_list = [], [], [], [], [], []

# 이미 수집한 상품 링크를 저장하여 중복을 방지하는 set
processed_links = set()

# 네이버 쇼핑의 일반적인 상품 카드 선택자
card_selector = 'div[class^="product_item_inner"]'

# 연속으로 새 상품을 찾지 못한 스크롤 횟수
empty_scroll_count = 0

# 최대 허용 연속 실패 횟수
max_empty_scrolls = 3

# 메인 루프 시작
while True:
    # 이번 수집 주기에서 새로운 상품을 찾았는지 기록하는 플래그
    found_new_product_in_cycle = False
    
    # WebDriverWait를 사용하여 상품 카드가 최소 10개 이상 로드될 때까지 기다림
    try:
        WebDriverWait(driver, 10).until(
            lambda d: len(d.find_elements(By.CSS_SELECTOR, card_selector)) >= 10
        )
        time.sleep(random.uniform(5, 10))
    except Exception:
        print("ℹ️ 페이지에서 상품 카드를 찾을 수 없거나 로딩 시간이 초과되었습니다.")

    # 현재 화면의 HTML을 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    current_cards = soup.select(card_selector)

    for card in current_cards:
        try:
            # 상품 링크를 더 정확하게 찾고, 광고 상품은 제외
            link_tag = card.select_one('a[href*="/products/"]')
            if not link_tag or 'ad' in link_tag.get('href', ''):
                continue
                
            link = link_tag['href']
            
            # 이전에 수집한 적 없는 새로운 상품인 경우에만 데이터 추출
            if link not in processed_links:
                found_new_product_in_cycle = True # ✨ 새로운 상품 발견! 플래그를 True로 변경
                processed_links.add(link)
                
                title = card.select_one('div[class^="product_title"]').get_text(strip=True) if card.select_one('div[class^="product_title"]') else 'N/A'
                store = card.select_one('a[class^="product_mall_link"]').get_text(strip=True) if card.select_one('a[class^="product_mall_link"]') else 'N/A'
                
                price_info = card.select_one('div[class^="product_price_area"]')
                original_price_tag = price_info.select_one('span[class*="price_original"]')
                original_price = int(original_price_tag.get_text(strip=True).replace('원', '').replace(',', '')) if original_price_tag else np.nan

                current_price_tag = price_info.select_one('span[class*="price_num"]')
                current_price = int(current_price_tag.get_text(strip=True).replace(',', '')) if current_price_tag else np.nan
                
                discount_tag = price_info.select_one('span[class*="price_discount"]')
                discount_rate = int(discount_tag.get_text(strip=True).replace('%', '')) / 100 if discount_tag else np.nan

                link_list.append(link)
                title_list.append(title)
                store_list.append(store)
                original_price_list.append(original_price)
                current_price_list.append(current_price)
                discount_rate_list.append(discount_rate)
                
                print(f"✅ [{len(processed_links)}] 목록 수집 성공: {title}")
        except Exception:
            continue
    
    if found_new_product_in_cycle:
        empty_scroll_count = 0 # 이번 주기에 새 상품을 발견했다면, 실패 횟수 초기화
    else:
        empty_scroll_count += 1 # 새 상품을 발견하지 못했다면, 실패 횟수 1 증가
        print(f"ℹ️ 이번 스크롤에서 새 상품을 찾지 못했습니다. (연속 실패: {empty_scroll_count}회)")

    if empty_scroll_count >= max_empty_scrolls: # 연속 실패 횟수가 최대 허용치에 도달하면 루프 종료
        print(f"\n{max_empty_scrolls}회 연속으로 새 상품이 없어 목록 수집을 종료합니다.")
        break
    
    # 로드된 페이지의 맨 아래로 스크롤
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 약 60초 동안 사람처럼 스크롤을 시도합니다.
    # human_like_scroll(driver, total_scroll_time=60)
    advanced_smart_scroll(driver)
    time.sleep(random.uniform(5, 10))

✅ [1] 수집 성공: 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개
✅ [2] 수집 성공: 릴리이브 탈모케어 정수리 M자완화 볼륨 샴푸
✅ [3] 수집 성공: 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개
✅ [4] 수집 성공: 러브밋 비건 저자극 탈모완화 볼륨 샴푸
✅ [5] 수집 성공: 라보에이치 탈모샴푸 약산성 대용량 두피 비건 400ml+400ml리필+180ml
✅ [6] 수집 성공: 려 루트젠 두피 에센스 대용량 145ml 두피 영양제 여성 남성 탈모 앰플
✅ [7] 수집 성공: 블루셀 마이크로바이옴 맥주효모 샴푸 알러젠프리향, 500ml, 1개
✅ [8] 수집 성공: 바이브랩 리바이 솔루션 안티 헤어 로스 스칼프 샴푸 우디플로럴머스크향, 500ml, 1개
✅ [9] 수집 성공: 려 루트젠 탈모샴푸 대용량 약산성 여성 비건 볼륨 두피 515ml+242ml+100ml
✅ [10] 수집 성공: 닥터포헤어 바이오3 폴리젠 탈모완화 샴푸 허브향, 500ml, 3개
✅ [11] 수집 성공: 네이처리퍼블릭 블랙빈 안티 헤어로스 샴푸(신형) 520ml, 2개
✅ [12] 수집 성공: 그래비티 엑스트라 스트롱 샴푸 475ml + 475ml(리필) + 30ml x 2 1개
✅ [13] 수집 성공: 메디큐브 소이민트 지루성두피 스케일링 비듬샴푸 490ml 2개
✅ [14] 수집 성공: 라보에이치 여름 쿨샴푸 지성 탈모 대용량 두피스케일링 400ml&400ml리필&180ml
✅ [15] 수집 성공: 반코르 맥주효모 탈모 완화 샴푸 프레쉬향, 500ml, 1개
✅ [16] 수집 성공: 1+1 살림백서 탈모 샴푸 엑티브B7 맥주효모 앤 비오틴 1000ml 남자 여자 바이오틴
✅ [17] 수집 성공: 메디큐브 소이민트 지루성두피 스케일링 비듬샴푸 490ml 2개
✅ [18] 수집 성공: 비건 탈모 샴푸 설페이트 계면활성제 없는 천연
✅ [19] 수집 성공: 그룬플러스 지성 쿨링 볼륨 탈모샴푸 250ml
✅ [20] 수집 

In [1]:
temp_df = pd.read_csv("(processing)naver_shopping_hair_loss_20250910.csv")
temp_df.info()

NameError: name 'pd' is not defined

In [ ]:
# --- 4. 상세 정보 수집 (수동 개입 및 재시도 기능 추가) ---
# '성분' 정보를 추가하기 위해 리스트 확장
manufacturer_list, brand_list, model_name_list, scalp_type_list, hair_type_list, type_list, product_form_list, features_list, ingredient_list = ([] for _ in range(9))

print(f"\n총 {len(link_list)}개의 상품 상세 정보 수집을 시작합니다.")
current_index = 0
pbar = tqdm(total=len(link_list), desc="상세 정보 수집 중")

while current_index < len(link_list):
    link = link_list[current_index]
    try:
        driver.get(link)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.DCVBehA8ZB'))
        )
        time.sleep(random.uniform(5, 10))
        
        # '성분'을 찾을 레이블에 추가
        labels_to_find = ["제조사", "브랜드", "모델명", "두피타입", "모발타입", "타입", "제품형태", "주요제품특징", "｢화장품법｣에 따라 기재 표시하여야 하는 모든성분"]
        detail_info = {label: np.nan for label in labels_to_find}
        # '성분'의 키 값 통일을 위해 별도 처리
        detail_info['성분'] = np.nan

        all_headers = driver.find_elements(By.CSS_SELECTOR, "table.RCLS1uAn0a th.rSg_SEReAx")

        for header in all_headers:
            header_text = header.text
            if header_text in labels_to_find:
                value_cell = header.find_element(By.XPATH, "./following-sibling::td")
                # '성분' 정보는 키 값을 통일하여 저장
                if '모든성분' in header_text:
                    detail_info['성분'] = value_cell.text
                else:
                    detail_info[header_text] = value_cell.text
        
        manufacturer_list.append(detail_info["제조사"])
        brand_list.append(detail_info["브랜드"])
        model_name_list.append(detail_info["모델명"])
        scalp_type_list.append(detail_info["두피타입"])
        hair_type_list.append(detail_info["모발타입"])
        type_list.append(detail_info["타입"])
        product_form_list.append(detail_info["제품형태"])
        features_list.append(detail_info["주요제품특징"])
        ingredient_list.append(detail_info["성분"]) 
        
        current_index += 1
        pbar.update(1)

    except Exception as e:
        print(f"\n⚠️ [{current_index + 1}번째 링크] 페이지 로딩 실패 또는 캡챠 감지.")
        print(f"   링크: {link}")
        input("▶ 브라우저에서 문제를 해결한 후, 터미널로 돌아와 Enter 키를 눌러주세요...")
        print(f"   [{current_index + 1}번째 링크] 재시도를 시작합니다.")
        continue

pbar.close()
print("\n✅ 모든 상품의 상세 정보 수집을 완료했습니다.")

In [ ]:
# --- 5. 최종 데이터프레임 생성 ---
df = pd.DataFrame({
    '상품명': title_list,
    '판매처': store_list,
    '정상가': original_price_list,
    '할인가': current_price_list,
    '할인율': discount_rate_list,
    '제조사': manufacturer_list,
    '브랜드': brand_list,
    '모델명': model_name_list,
    '두피타입': scalp_type_list,
    '모발타입': hair_type_list,
    '타입': type_list,
    '제품형태': product_form_list,
    '주요특징': features_list,
    '성분': ingredient_list, 
    '링크': link_list
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   판매처     1900 non-null   object 
 2   정상가     1051 non-null   float64
 3   할인가     1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   제조사     1510 non-null   object 
 6   브랜드     1520 non-null   object 
 7   모델명     897 non-null    object 
 8   두피타입    710 non-null    object 
 9   모발타입    699 non-null    object 
 10  타입      749 non-null    object 
 11  제품형태    670 non-null    object 
 12  주요특징    898 non-null    object 
 13  링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 207.9+ KB


In [ ]:
# --- 6. 자동 재시도 루프 ---

# 재시도 횟수 설정
MAX_RETRIES = 3

for i in range(MAX_RETRIES):
    # '브랜드' 정보가 비어있는(수집 실패한) 행을 찾습니다.
    failed_df = df[df['브랜드'].isnull()]
    
    # 만약 실패한 항목이 더 이상 없으면 재시도 루프를 종료합니다.
    if failed_df.empty:
        print("\n✅ 모든 항목 수집 완료. 재시도를 종료합니다.")
        break
        
    print(f"\n--- 재시도 {i+1}/{MAX_RETRIES} | 남은 항목: {len(failed_df)}개 ---")
    
    # 실패한 행들을 순회하며 재수집
    for index, row in tqdm(failed_df.iterrows(), total=len(failed_df), desc=f"재시도 {i+1}"):
        link = row['링크']
        try:
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.DCVBehA8ZB'))
            )
            time.sleep(random.uniform(5, 10))

            # (1차 수집과 동일한 정보 추출 로직)
            labels_to_find = [
                "제조사", "브랜드", "모델명", "두피타입", "모발타입",
                "타입", "제품형태", "성분", "주요제품특징"
            ]
            detail_info = {}
            all_headers = driver.find_elements(By.CSS_SELECTOR, "table.RCLS1uAn0a th.rSg_SEReAx")
            
            for header in all_headers:
                header_text = header.text
                if header_text in labels_to_find:
                    value_cell = header.find_element(By.XPATH, "./following-sibling::td")
                    detail_info[header_text] = value_cell.text

            # ⚠️ df.loc를 사용해 원본 데이터프레임의 해당 위치에 값을 '업데이트'합니다.
            df.loc[index, '제조사'] = detail_info.get("제조사", np.nan)
            df.loc[index, '브랜드'] = detail_info.get("브랜드", np.nan)
            df.loc[index, '모델명'] = detail_info.get("모델명", np.nan)
            df.loc[index, '두피타입'] = detail_info.get("두피타입", np.nan)
            df.loc[index, '모발타입'] = detail_info.get("모발타입", np.nan)
            df.loc[index, '타입'] = detail_info.get("타입", np.nan)
            df.loc[index, '제품형태'] = detail_info.get("제품형태", np.nan)
            df.loc[index, '성분'] = detail_info.get("성분", np.nan)
            df.loc[index, '주요특징'] = detail_info.get("주요제품특징", np.nan)
            
        except Exception:
            # 재시도도 실패하면 그냥 넘어갑니다. (값은 여전히 NaN으로 유지)
            continue


--- 재시도 1/3 | 남은 항목: 427개 ---


재시도 1: 100%|██████████| 427/427 [16:36<00:00,  2.33s/it] 



--- 재시도 2/3 | 남은 항목: 381개 ---


재시도 2: 100%|██████████| 381/381 [14:40<00:00,  2.31s/it] 



--- 재시도 3/3 | 남은 항목: 380개 ---


재시도 3: 100%|██████████| 380/380 [14:52<00:00,  2.35s/it] 


In [ ]:
# --- 7. 최종 결과 확인 ---
print("\n--- 최종 수집 결과 ---")
print(df.info()) # 각 컬럼별 비어있지 않은 데이터 수를 확인


--- 최종 수집 결과 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   판매처     1900 non-null   object 
 2   정상가     1051 non-null   float64
 3   할인가     1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   제조사     1510 non-null   object 
 6   브랜드     1520 non-null   object 
 7   모델명     897 non-null    object 
 8   두피타입    710 non-null    object 
 9   모발타입    699 non-null    object 
 10  타입      749 non-null    object 
 11  제품형태    670 non-null    object 
 12  주요특징    898 non-null    object 
 13  링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 207.9+ KB
None


In [ ]:
# [processing]naver_shopping_hair_loss_20250910 : 현재까지 수집한 데이터
df.to_csv("네이버쇼핑_탈모샴푸_크롤링결과.csv", index=False, encoding='utf-8-sig')
print("\n'네이버쇼핑_탈모샴푸_크롤링결과.csv' 파일로 저장을 완료했습니다.")

In [6]:
# 드라이버 종료
driver.quit()